# 📊 PyPortfolio - Otimização de Portfólio v2.6**Markowitz, Fronteira Eficiente, Monte Carlo e Sharpe vs CDI**---## 📋 Metadados da Versão| Campo | Valor ||-------|-------|| **Versão** | v2.6 || **Data** | Janeiro/2026 || **Autor** | PyPortfolio Team |### Changelog v2.6- ✅ **P0: Seletor de arquivos corrigido** - Sempre lista todos os arquivos, permite escolha mesmo com arquivo padrão- ✅ **P0: Documentação de parâmetros** - Tabela detalhada explicando cada configuração- ✅ **P1: Pergunta sobre carregar pesos** - Controle explícito do usuário- ✅ **P1: Auditorias reforçadas** - Alertas quando cobertura < 60%- ✅ **Roadmap priorizado** no final do notebook### Mudanças desde v2.5:- Seletor de arquivos reescrito com varredura completa- Seção de documentação de parâmetros adicionada- Input interativo para carregar carteira atual- Alertas de cobertura baixa implementados

---## 📖 Parâmetros e Configuração (Documentação)### Parâmetros de Qualidade de Dados| Parâmetro | Valor Padrão | O que controla | Quando ajustar | Impacto / Trade-off ||-----------|--------------|----------------|----------------|---------------------|| `MIN_OBS` | 12 | Mínimo de meses de histórico por ativo | Se ativos novos estão sendo excluídos desnecessariamente | Valor baixo: inclui ativos com pouco histórico (mais ruído). Valor alto: exclui ativos novos mas promissores || `MIN_OVERLAP` | 12 | Mínimo de meses em comum entre pares para calcular covariância | Se muitas covariâncias estão sendo zeradas | Valor baixo: covariâncias menos confiáveis. Valor alto: mais pares zerados, matriz mais esparsa || `VOL_FLOOR_MENSAL` | 0.005 (0.5%) | Piso de volatilidade mensal para evitar vol~0 | Se ativos de renda fixa dominam o GMV artificialmente | Valor baixo: GMV pode ter vol artificial. Valor alto: penaliza injustamente ativos estáveis |### Parâmetros de Mapeamento| Parâmetro | Valor Padrão | O que controla | Quando ajustar | Impacto / Trade-off ||-----------|--------------|----------------|----------------|---------------------|| `THRESHOLD_OK` | 0.90 | Score mínimo para aceitar mapeamento automaticamente | Se mapeamentos corretos estão sendo rejeitados | Valor baixo: mais falsos positivos. Valor alto: mais revisões manuais || `THRESHOLD_REVISAR` | 0.75 | Score mínimo para aceitar com flag "REVISAR" | Se muitos ativos não estão sendo mapeados | Valor baixo: aceita matches ruins. Valor alto: mais ativos ficam sem mapeamento || `ATIVOS_EXCLUIR` | ["USDC"] | Lista de ativos a remover do universo | Quando stablecoins ou outros ativos distorcem a otimização | Excluir demais: perde diversificação. Incluir demais: distorce GMV |### Parâmetros de Monte Carlo| Parâmetro | Valor Padrão | O que controla | Quando ajustar | Impacto / Trade-off ||-----------|--------------|----------------|----------------|---------------------|| `NUM_PORTFOLIOS` | 80000 | Quantidade de carteiras simuladas | Se a nuvem está rala ou processamento muito lento | Mais: melhor cobertura, mais lento. Menos: mais rápido, nuvem incompleta || `MC_ALPHA_CONC` | 0.2 | Parâmetro Dirichlet para carteiras concentradas | Se quer mais/menos carteiras concentradas na nuvem | Menor: mais concentrado. Maior: mais diversificado || `MC_ALPHA_DIV` | 1.0 | Parâmetro Dirichlet para carteiras diversificadas | Geralmente não precisa ajustar | 1.0 = distribuição uniforme no simplex || `MC_FRAC_CANTOS` | 0.05 (5%) | Fração de carteiras "quase 100% em 1 ativo" | Se quer explorar mais os extremos | Mais: nuvem cobre melhor os cantos || `MC_FRAC_SPARSE` | 0.15 (15%) | Fração de carteiras com poucos ativos (3-8) | Se quer carteiras mais práticas | Mais: mais carteiras implementáveis na prática |### Parâmetros da Fronteira Eficiente| Parâmetro | Valor Padrão | O que controla | Quando ajustar | Impacto / Trade-off ||-----------|--------------|----------------|----------------|---------------------|| `N_PONTOS_FRONTEIRA` | 60 | Quantidade de pontos na curva da fronteira | Se a curva está "pixelada" ou processamento lento | Mais: curva mais suave, mais lento |### Parâmetros de Taxa Livre de Risco| Parâmetro | Valor Padrão | O que controla | Quando ajustar | Impacto / Trade-off ||-----------|--------------|----------------|----------------|---------------------|| `RF_MODO` | "media_periodo" | Como obter o CDI | Se quer CDI atual vs histórico vs manual | "atual": CDI de hoje. "media_periodo": CDI médio do período dos dados. "manual": valor fixo || `RF_MANUAL` | 0.1150 (11.5%) | Taxa manual se RF_MODO="manual" | Se API do BCB não funciona ou quer valor específico | - || `RF_FALLBACK` | 0.10 (10%) | Taxa de fallback se tudo falhar | Raramente precisa ajustar | - |### Parâmetros de Otimização| Parâmetro | Valor Padrão | O que controla | Quando ajustar | Impacto / Trade-off ||-----------|--------------|----------------|----------------|---------------------|| `PESO_MIN_ATIVO` | 0.0 | Peso mínimo por ativo (long-only) | Se quiser permitir short | 0.0 = long-only. <0 = permite short || `PESO_MAX_ATIVO` | 1.0 | Peso máximo por ativo | Se quiser limitar concentração | <1.0 força diversificação mínima || `USAR_MEDIA_GEOMETRICA` | True | Usar retorno geométrico vs aritmético | Se prefere estimativa mais otimista (aritmética) | Geométrica: mais conservadora e realista |

---## 🔧 BLOCO 0 - CONFIGURAÇÕES GLOBAIS

In [ ]:
# =============================================================================# METADADOS DA VERSÃO# =============================================================================VERSAO = "v2.6"DATA_VERSAO = "Janeiro/2026"MUDANCAS_VERSAO = [    "P0: Seletor de arquivos corrigido - lista todos, permite escolha",    "P0: Documentação de parâmetros em tabela",    "P1: Pergunta explícita sobre carregar pesos",    "P1: Alertas de cobertura baixa (< 60%)",    "Roadmap priorizado no final"]print(f"📊 PyPortfolio {VERSAO} ({DATA_VERSAO})")print("   Mudanças nesta versão:")for m in MUDANCAS_VERSAO:    print(f"   • {m}")

In [ ]:
# =============================================================================# CONFIGURAÇÕES GLOBAIS - EDITE AQUI# =============================================================================# --- Caminhos de Arquivos ---# Aceita tanto "/" (Linux/Mac) quanto "\" (Windows)EXCEL_DIR = "1 - Dados/1 - Rentabilidade atual"EXCEL_PATTERN = "*.xlsx"  # Padrão amplo para encontrar todos os ExcelARQUIVO_PADRAO = "Rendimentos_Mensais_Ativos_v7.0.xlsx"  # Nome esperado# --- Período de Análise ---DATA_INICIO = "2020-01-01"DATA_FIM = "2026-01-01"# --- Parâmetros de Qualidade de Dados ---MIN_OBS = 12              # Mínimo de meses por ativoMIN_OVERLAP = 12          # Mínimo de meses em comum para covariânciaVOL_FLOOR_MENSAL = 0.005  # 0.5% ao mês (~1.7% a.a.)# --- Ativos a excluir do universo ---ATIVOS_EXCLUIR = ["USDC"]  # Stablecoins distorcem GMV# --- Thresholds de mapeamento ---THRESHOLD_OK = 0.90       # score >= 0.90: aceita automaticamenteTHRESHOLD_REVISAR = 0.75  # 0.75 <= score < 0.90: aceita mas marca REVISARALERTA_COBERTURA_MINIMA = 0.60  # Alerta se cobertura < 60%# --- Taxa Livre de Risco ---RF_MODO = "media_periodo"  # "atual" | "media_periodo" | "manual"RF_MANUAL = 0.1150         # 11.5% a.a.RF_FALLBACK = 0.10         # 10% a.a.# --- Parâmetros de Otimização ---PESO_MAX_ATIVO = 1.0PESO_MIN_ATIVO = 0.0# --- Monte Carlo ---NUM_PORTFOLIOS = 80000RANDOM_SEED = 42MC_ALPHA_CONC = 0.2       # Dirichlet concentradoMC_ALPHA_DIV = 1.0        # Dirichlet diversificadoMC_FRAC_CANTOS = 0.05     # 5% carteiras quase-100% em 1 ativoMC_FRAC_SPARSE = 0.15     # 15% carteiras sparse (3-8 ativos)# --- Fronteira Eficiente ---N_PONTOS_FRONTEIRA = 60# --- Retorno ---USAR_MEDIA_GEOMETRICA = True# --- Visualização ---TOP_N_PESOS = 20print("\n✅ Configurações carregadas!")print(f"   Diretório: {EXCEL_DIR}")print(f"   MIN_OBS: {MIN_OBS} | MIN_OVERLAP: {MIN_OVERLAP} | VOL_FLOOR: {VOL_FLOOR_MENSAL*100:.1f}%")print(f"   Alerta cobertura: < {ALERTA_COBERTURA_MINIMA*100:.0f}%")

---## 📦 BLOCO 1 - BIBLIOTECAS E FUNÇÕES UTILITÁRIAS

In [ ]:
# =============================================================================# IMPORTS# =============================================================================import pandas as pdimport numpy as npfrom scipy.optimize import minimizefrom scipy.linalg import eighimport plotly.graph_objects as goimport warningsfrom pathlib import Pathfrom difflib import SequenceMatcherimport reimport osfrom typing import Optional, Dict, Tuple, Listfrom IPython.display import displayfrom datetime import datetimetry:    import yfinance as yf    YFINANCE_DISPONIVEL = Trueexcept ImportError:    YFINANCE_DISPONIVEL = Falsewarnings.filterwarnings('ignore', category=FutureWarning)pd.set_option('display.max_columns', 20)pd.set_option('display.max_rows', 50)print("✅ Bibliotecas importadas!")

In [ ]:
# =============================================================================# FUNÇÕES - BUSCA E SELEÇÃO DE ARQUIVO EXCEL (CORRIGIDO v2.6)# =============================================================================def varrer_arquivos_excel(diretorio: str, pattern: str = "*.xlsx", recursive: bool = False) -> List[Dict]:    """    Varre o diretório buscando arquivos Excel.        Regras:    - Aceita .xlsx e .XLSX    - Ignora arquivos temporários do Excel (~$...)    - Retorna lista de dicts com metadados    """    dir_path = Path(diretorio)        # Verificar se diretório existe    if not dir_path.exists():        print(f"⚠️ Diretório não encontrado: {diretorio}")        # Tentar variações de caminho (Windows vs Linux)        alternativas = [            diretorio.replace("/", "\\"),            diretorio.replace("\\", "/"),            str(Path(diretorio).absolute())        ]        for alt in alternativas:            if Path(alt).exists():                dir_path = Path(alt)                print(f"   Usando alternativa: {alt}")                break        else:            return []        # Buscar arquivos    if recursive:        arquivos = list(dir_path.rglob(pattern))    else:        arquivos = list(dir_path.glob(pattern))        # Também buscar .XLSX (case insensitive)    if pattern.endswith(".xlsx"):        pattern_upper = pattern.replace(".xlsx", ".XLSX")        if recursive:            arquivos.extend(dir_path.rglob(pattern_upper))        else:            arquivos.extend(dir_path.glob(pattern_upper))        # Filtrar e processar    resultado = []    for arq in arquivos:        nome = arq.name                # Ignorar arquivos temporários do Excel        if nome.startswith("~$"):            continue                # Ignorar arquivos ocultos        if nome.startswith("."):            continue                # Obter metadados        try:            stat = arq.stat()            mtime = datetime.fromtimestamp(stat.st_mtime)            tamanho_kb = stat.st_size / 1024                        resultado.append({                'path': arq,                'nome': nome,                'modificado': mtime,                'tamanho_kb': tamanho_kb,                'modificado_str': mtime.strftime("%Y-%m-%d %H:%M")            })        except Exception as e:            print(f"   ⚠️ Erro ao ler {nome}: {e}")        # Ordenar por data de modificação (mais recente primeiro)    resultado.sort(key=lambda x: x['modificado'], reverse=True)        return resultadodef selecionar_arquivo_excel_interativo(    diretorio: str,     pattern: str = "*.xlsx",    arquivo_padrao: str = None,    recursive: bool = False) -> str:    """    Seleciona arquivo Excel com interação SEMPRE oferecida ao usuário.        Fluxo:    1. Varre TODOS os arquivos na pasta    2. Se arquivo padrão existe, pergunta se quer usar    3. Se não, lista todos e permite escolha    """    print("=" * 60)    print("SELEÇÃO DE ARQUIVO EXCEL")    print("=" * 60)        # SEMPRE fazer varredura completa primeiro    arquivos = varrer_arquivos_excel(diretorio, pattern, recursive)        if not arquivos:        print(f"\n❌ ERRO: Nenhum arquivo Excel encontrado!")        print(f"   Diretório: {diretorio}")        print(f"   Padrão: {pattern}")        print(f"\n   Verifique se:")        print(f"   1. O diretório existe e está correto")        print(f"   2. Há arquivos .xlsx na pasta")        print(f"   3. Os arquivos não estão abertos no Excel (temporários ~$)")        raise FileNotFoundError(f"Nenhum arquivo Excel em {diretorio}")        # Mostrar resumo    print(f"\n📂 Encontrados {len(arquivos)} arquivo(s) Excel em: {diretorio}")    print("-" * 60)        # Verificar se arquivo padrão existe    caminho_padrao = None    idx_padrao = None    if arquivo_padrao:        for i, arq in enumerate(arquivos):            if arq['nome'] == arquivo_padrao:                caminho_padrao = str(arq['path'])                idx_padrao = i                break        # Listar TODOS os arquivos    for i, arq in enumerate(arquivos):        marcador = " ⭐ (padrão)" if i == idx_padrao else ""        print(f"   [{i}] {arq['nome']}{marcador}")        print(f"       Modificado: {arq['modificado_str']} | Tamanho: {arq['tamanho_kb']:.1f} KB")        # Decidir fluxo    if caminho_padrao and len(arquivos) == 1:        # Único arquivo = usar direto        print(f"\n✅ Único arquivo encontrado (é o padrão)")        return caminho_padrao        if caminho_padrao:        # Arquivo padrão existe, mas há outros - perguntar        print(f"\n📌 Arquivo padrão encontrado: {arquivo_padrao}")        try:            usar_padrao = input("   Usar este arquivo? (s/n) [s]: ").strip().lower()            if usar_padrao == "" or usar_padrao == "s":                print(f"\n✅ Selecionado: {arquivo_padrao}")                return caminho_padrao        except:            # Modo não-interativo            print("   (modo não-interativo: usando padrão)")            return caminho_padrao        # Permitir escolha por índice    print(f"\n   [ENTER] = usar mais recente ({arquivos[0]['nome']})")        try:        escolha = input("   Digite o número do arquivo desejado: ").strip()        if escolha == "":            idx = 0        else:            idx = int(escolha)    except:        idx = 0        print("   (modo não-interativo: usando mais recente)")        # Validar índice    if 0 <= idx < len(arquivos):        selecionado = arquivos[idx]    else:        print(f"   ⚠️ Índice {idx} inválido, usando mais recente")        selecionado = arquivos[0]        print(f"\n✅ Selecionado: {selecionado['nome']}")    return str(selecionado['path'])print("✅ Funções de seleção de arquivo carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - LEITURA E PROCESSAMENTO DE DADOS# =============================================================================def normalizar_nome(nome: str) -> str:    """Normaliza nome de ativo para comparação."""    if pd.isna(nome):        return ""    nome = str(nome).lower().strip()    acentos = {'á':'a','à':'a','ã':'a','â':'a','é':'e','ê':'e','í':'i',               'ó':'o','ô':'o','õ':'o','ú':'u','ç':'c'}    for k, v in acentos.items():        nome = nome.replace(k, v)    nome = re.sub(r'[-/_]', ' ', nome)    nome = re.sub(r'[^a-z0-9\s]', ' ', nome)    nome = re.sub(r'\s+', ' ', nome).strip()    return nomedef similaridade(a: str, b: str) -> float:    """Retorna score de similaridade entre 0 e 1."""    na, nb = normalizar_nome(a), normalizar_nome(b)    score = SequenceMatcher(None, na, nb).ratio()        if na in nb or nb in na:        score = max(score, 0.85)        words_a = set(na.split())    words_b = set(nb.split())    common = words_a & words_b    if len(common) >= 2:        score = max(score, 0.75 + 0.05 * len(common))        return min(score, 1.0)def parse_percentual(valor) -> float:    """Converte valor percentual para float decimal."""    if pd.isna(valor) or valor == '-' or valor == '':        return np.nan    if isinstance(valor, (int, float)):        if abs(valor) > 1:            return valor / 100        return valor    valor = str(valor).strip().replace('%', '').replace(',', '.')    try:        return float(valor) / 100    except:        return np.nandef ler_retornos_aba_excel(xlsx_path: str, sheet_name: str) -> pd.Series:    """Lê retornos mensais de uma aba do Excel."""    df = pd.read_excel(xlsx_path, sheet_name=sheet_name, header=None, engine='openpyxl')        header_row = None    for i, row in df.iterrows():        if str(row.iloc[0]).strip().lower() == 'ano':            header_row = i            break        if header_row is None:        raise ValueError(f"Cabeçalho 'Ano' não encontrado na aba '{sheet_name}'")        meses_map = {'jan':1,'fev':2,'mar':3,'abr':4,'mai':5,'jun':6,                 'jul':7,'ago':8,'set':9,'out':10,'nov':11,'dez':12}        header = [str(c).strip().lower()[:3] for c in df.iloc[header_row].values]        retornos = []    for i in range(header_row + 1, len(df)):        row = df.iloc[i]        ano_val = row.iloc[0]                if pd.isna(ano_val) or str(ano_val).strip().lower() in ['', 'estatísticas', 'estatisticas', 'meses']:            break                try:            ano = int(ano_val)        except:            break                for col_idx, col_name in enumerate(header[1:], start=1):            if col_name in meses_map:                mes = meses_map[col_name]                val = parse_percentual(row.iloc[col_idx])                if not pd.isna(val):                    data = pd.Timestamp(year=ano, month=mes, day=1)                    retornos.append({'data': data, 'retorno': val})        if not retornos:        return pd.Series(dtype=float)        df_ret = pd.DataFrame(retornos).set_index('data').sort_index()    return df_ret['retorno']def ler_pesos_resumo_robusto(xlsx_path: str) -> Tuple[Dict[str, float], str]:    """Lê pesos da aba 'Resumo' com busca robusta da coluna."""    try:        df = pd.read_excel(xlsx_path, sheet_name='Resumo', header=None, engine='openpyxl')    except Exception as e:        return {}, f"erro: {str(e)[:50]}"        header_row = None    col_ativo = None    col_peso = None        keywords_peso = ['peso', 'weight', 'alocacao', 'alocação']    keywords_carteira = ['cart', 'portfolio', 'atual', 'current']        for i, row in df.iterrows():        for j, val in enumerate(row):            if pd.isna(val):                continue            val_str = str(val).strip().lower()                        if val_str == 'ativo' or val_str == 'asset':                col_ativo = j                        has_peso = any(kw in val_str for kw in keywords_peso)            has_cart = any(kw in val_str for kw in keywords_carteira)            if has_peso and has_cart:                col_peso = j                if col_ativo is not None and col_peso is not None:            header_row = i            break        if col_peso is None:        return {}, "nao_encontrado"        print(f"   ✅ Coluna de pesos: '{df.iloc[header_row, col_peso]}' (linha {header_row})")        pesos = {}    for i in range(header_row + 1, len(df)):        row = df.iloc[i]        ativo = row.iloc[col_ativo]        peso = row.iloc[col_peso]                if pd.isna(ativo) or str(ativo).strip() == '':            continue                ativo = str(ativo).strip()        try:            peso_val = float(peso) if not pd.isna(peso) else 0.0            if peso_val > 1:                peso_val = peso_val / 100            pesos[ativo] = peso_val        except:            pesos[ativo] = 0.0        return pesos, "encontrado"print("✅ Funções de leitura carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - MAPEAMENTO DE PESOS# =============================================================================def mapear_pesos_para_abas(    pesos_resumo: Dict[str, float],    abas_candidatas: List[str],    abas_excluidas_min_obs: List[str],    threshold_ok: float = 0.90,    threshold_revisar: float = 0.75) -> pd.DataFrame:    """    Mapeia ativos do Resumo para abas do Excel ANTES de aplicar MIN_OBS.    """    resultados = []    todas_abas = set(abas_candidatas) | set(abas_excluidas_min_obs)    abas_excluidas_set = set(abas_excluidas_min_obs)        for ativo_resumo, peso in pesos_resumo.items():        if peso == 0:            continue                # Match exato        if ativo_resumo in todas_abas:            if ativo_resumo in abas_excluidas_set:                status = "MAPEADO_EXCLUIDO_MIN_OBS"                motivo = f"Aba '{ativo_resumo}' excluída por MIN_OBS"            else:                status = "OK"                motivo = "Match exato"                        resultados.append({                'ativo_resumo': ativo_resumo, 'peso': peso,                'aba_mapeada': ativo_resumo, 'score': 1.0,                'status': status, 'motivo': motivo            })            continue                # Fuzzy match        melhor_match = None        melhor_score = 0        for aba in todas_abas:            score = similaridade(ativo_resumo, aba)            if score > melhor_score:                melhor_score = score                melhor_match = aba                if melhor_score >= threshold_ok:            if melhor_match in abas_excluidas_set:                status = "MAPEADO_EXCLUIDO_MIN_OBS"                motivo = f"Aba '{melhor_match}' excluída por MIN_OBS"            else:                status = "OK"                motivo = f"Fuzzy match (score={melhor_score:.2f})"        elif melhor_score >= threshold_revisar:            if melhor_match in abas_excluidas_set:                status = "MAPEADO_EXCLUIDO_MIN_OBS"                motivo = f"Aba '{melhor_match}' excluída por MIN_OBS (revisar)"            else:                status = "REVISAR"                motivo = f"Score médio ({melhor_score:.2f})"        else:            status = "NAO_MAPEADO"            melhor_match = None            motivo = f"Melhor score ({melhor_score:.2f}) abaixo do threshold"                resultados.append({            'ativo_resumo': ativo_resumo, 'peso': peso,            'aba_mapeada': melhor_match, 'score': melhor_score,            'status': status, 'motivo': motivo        })        return pd.DataFrame(resultados)def gerar_relatorio_mapeamento(df_mapa: pd.DataFrame, alerta_minimo: float = 0.60) -> Dict:    """Gera resumo numérico do mapeamento com alertas."""    peso_total = df_mapa['peso'].sum()        peso_ok = df_mapa[df_mapa['status'] == 'OK']['peso'].sum()    peso_revisar = df_mapa[df_mapa['status'] == 'REVISAR']['peso'].sum()    peso_excluido = df_mapa[df_mapa['status'] == 'MAPEADO_EXCLUIDO_MIN_OBS']['peso'].sum()    peso_nao_mapeado = df_mapa[df_mapa['status'] == 'NAO_MAPEADO']['peso'].sum()        peso_no_universo = peso_ok + peso_revisar    cobertura = peso_no_universo / peso_total if peso_total > 0 else 0        relatorio = {        'peso_total_resumo': peso_total,        'peso_mapeado_ok': peso_ok,        'peso_mapeado_revisar': peso_revisar,        'peso_mapeado_no_universo': peso_no_universo,        'peso_mapeado_excluido_min_obs': peso_excluido,        'peso_nao_mapeado': peso_nao_mapeado,        'cobertura': cobertura,        'alerta_cobertura_baixa': cobertura < alerta_minimo,        'n_ativos_ok': (df_mapa['status'] == 'OK').sum(),        'n_ativos_revisar': (df_mapa['status'] == 'REVISAR').sum(),        'n_ativos_excluidos': (df_mapa['status'] == 'MAPEADO_EXCLUIDO_MIN_OBS').sum(),        'n_ativos_nao_mapeados': (df_mapa['status'] == 'NAO_MAPEADO').sum()    }        return relatoriodef exibir_auditoria_mapeamento(relatorio: Dict, renormalizou: bool = False, base_renorm: float = 0):    """Exibe auditoria detalhada do mapeamento (P1)."""    print("\n" + "=" * 60)    print("🔍 AUDITORIA DE MAPEAMENTO")    print("=" * 60)        print(f"\n   Soma de pesos no Resumo:        {relatorio['peso_total_resumo']:.2%}")    print(f"   % Mapeado OK no universo:       {relatorio['peso_mapeado_ok']:.2%} ({relatorio['n_ativos_ok']} ativos)")    print(f"   % Mapeado REVISAR no universo:  {relatorio['peso_mapeado_revisar']:.2%} ({relatorio['n_ativos_revisar']} ativos)")    print(f"   % Total no universo:            {relatorio['peso_mapeado_no_universo']:.2%}")    print(f"   % Excluído por MIN_OBS:         {relatorio['peso_mapeado_excluido_min_obs']:.2%} ({relatorio['n_ativos_excluidos']} ativos)")    print(f"   % Não mapeado:                  {relatorio['peso_nao_mapeado']:.2%} ({relatorio['n_ativos_nao_mapeados']} ativos)")    print(f"\n   COBERTURA EFETIVA:              {relatorio['cobertura']:.1%}")        if renormalizou:        print(f"\n   ⚠️ Renormalizado: SIM (base: {base_renorm:.2%})")    else:        print(f"\n   Renormalizado: NÃO")        # ALERTA se cobertura baixa    if relatorio['alerta_cobertura_baixa']:        print("\n" + "!" * 60)        print("⚠️  ALERTA: COBERTURA MUITO BAIXA!")        print(f"   Apenas {relatorio['cobertura']:.1%} dos pesos foram mapeados para o universo.")        print("   Isso pode indicar:")        print("   • Muitos ativos excluídos por MIN_OBS")        print("   • Nomes no Resumo muito diferentes das abas")        print("   • Ativos novos ainda sem histórico suficiente")        print("\n   RECOMENDAÇÃO: Revise o mapeamento manualmente ou ajuste MIN_OBS.")        print("!" * 60)print("✅ Funções de mapeamento carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - CARREGAMENTO EM ETAPAS# =============================================================================def carregar_metadados_excel(xlsx_path: str, ativos_excluir: List[str] = None):    """ETAPA 1: Carrega lista de abas e contagem de meses."""    if ativos_excluir is None:        ativos_excluir = []        xlsx = pd.ExcelFile(xlsx_path, engine='openpyxl')    abas = xlsx.sheet_names        metadados = {        'total_abas': len(abas),        'abas_candidatas': [],        'meses_por_ativo': {},        'ativos_excluidos_manual': [],        'erros_leitura': []    }        for aba in abas:        if aba.lower() == 'resumo':            continue                if aba.upper() in [a.upper() for a in ativos_excluir]:            metadados['ativos_excluidos_manual'].append(aba)            continue                try:            serie = ler_retornos_aba_excel(xlsx_path, aba)            n_obs = len(serie.dropna())            metadados['meses_por_ativo'][aba] = n_obs            metadados['abas_candidatas'].append(aba)        except Exception as e:            metadados['erros_leitura'].append((aba, str(e)[:50]))        return metadadosdef filtrar_por_min_obs(metadados: Dict, min_obs: int) -> Tuple[List[str], List[str]]:    """ETAPA 2: Separa abas por MIN_OBS."""    abas_ok = []    abas_excluidas = []        for aba, n_meses in metadados['meses_por_ativo'].items():        if n_meses >= min_obs:            abas_ok.append(aba)        else:            abas_excluidas.append(aba)        return abas_ok, abas_excluidasdef carregar_retornos_das_abas(xlsx_path: str, abas: List[str]) -> pd.DataFrame:    """ETAPA 3: Carrega retornos das abas selecionadas."""    series_list = {}    for aba in abas:        serie = ler_retornos_aba_excel(xlsx_path, aba)        series_list[aba] = serie        return pd.DataFrame(series_list)print("✅ Funções de carregamento carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - ESTATÍSTICAS E MATRIZ DE COVARIÂNCIA# =============================================================================def retorno_geometrico_anual(retornos_mensais: pd.Series) -> float:    ret = retornos_mensais.dropna()    if len(ret) == 0:        return np.nan    prod = np.prod(1 + ret)    return float(prod ** (12 / len(ret)) - 1)def retorno_aritmetico_anual(retornos_mensais: pd.Series) -> float:    return float(retornos_mensais.mean() * 12)def volatilidade_anual(retornos_mensais: pd.Series) -> float:    return float(retornos_mensais.std() * np.sqrt(12))def calcular_matriz_overlap(retornos_df: pd.DataFrame) -> pd.DataFrame:    """Calcula matriz de overlaps entre pares de ativos."""    n = len(retornos_df.columns)    ativos = retornos_df.columns    overlap = pd.DataFrame(index=ativos, columns=ativos, dtype=float)        for i, ativo_i in enumerate(ativos):        for j, ativo_j in enumerate(ativos):            if i == j:                overlap.iloc[i, j] = len(retornos_df[ativo_i].dropna())            else:                mask = retornos_df[ativo_i].notna() & retornos_df[ativo_j].notna()                overlap.iloc[i, j] = mask.sum()        return overlapdef aplicar_min_overlap_covariancia(cov_matrix: pd.DataFrame, overlap_matrix: pd.DataFrame,                                     min_overlap: int) -> Tuple[pd.DataFrame, int]:    """Zera covariâncias de pares com overlap baixo."""    cov_ajustada = cov_matrix.copy()    pares_afetados = 0        for i, ativo_i in enumerate(cov_matrix.index):        for j, ativo_j in enumerate(cov_matrix.columns):            if i != j:                if overlap_matrix.iloc[i, j] < min_overlap:                    cov_ajustada.iloc[i, j] = 0                    cov_ajustada.iloc[j, i] = 0                    pares_afetados += 1        return cov_ajustada, pares_afetados // 2def aplicar_vol_floor_covariancia(cov_matrix: pd.DataFrame, vol_floor_mensal: float):    cov_floor_anual = (vol_floor_mensal ** 2) * 12        cov_ajustada = cov_matrix.copy()    ativos_ajustados = []        for i, ativo in enumerate(cov_matrix.index):        if cov_matrix.iloc[i, i] < cov_floor_anual:            cov_ajustada.iloc[i, i] = cov_floor_anual            ativos_ajustados.append(ativo)        return cov_ajustada, ativos_ajustadosdef corrigir_matriz_psd(cov_matrix: pd.DataFrame, epsilon: float = 1e-8):    arr = cov_matrix.values    eigenvalues, eigenvectors = eigh(arr)        if np.min(eigenvalues) >= -epsilon:        return cov_matrix, False        eigenvalues_corrigidos = np.maximum(eigenvalues, epsilon)    arr_corrigido = eigenvectors @ np.diag(eigenvalues_corrigidos) @ eigenvectors.T    arr_corrigido = (arr_corrigido + arr_corrigido.T) / 2        return pd.DataFrame(arr_corrigido, index=cov_matrix.index, columns=cov_matrix.columns), Truedef calcular_estatisticas_portfolio(retornos_df: pd.DataFrame, usar_geometrico: bool = True,                                    vol_floor_mensal: float = 0.0, min_overlap: int = 0):    if usar_geometrico:        retornos_esperados = retornos_df.apply(retorno_geometrico_anual)    else:        retornos_esperados = retornos_df.apply(retorno_aritmetico_anual)        cov_matrix = retornos_df.cov() * 12        pares_zerados = 0    if min_overlap > 0:        overlap_matrix = calcular_matriz_overlap(retornos_df)        cov_matrix, pares_zerados = aplicar_min_overlap_covariancia(cov_matrix, overlap_matrix, min_overlap)        ativos_floor = []    if vol_floor_mensal > 0:        cov_matrix, ativos_floor = aplicar_vol_floor_covariancia(cov_matrix, vol_floor_mensal)        return retornos_esperados, cov_matrix, pares_zerados, ativos_floorprint("✅ Funções de estatística carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - OTIMIZAÇÃO# =============================================================================class ResultadoOtimizacao:    def __init__(self, pesos, sucesso, mensagem, metodo):        self.pesos = pesos        self.sucesso = sucesso        self.mensagem = mensagem        self.metodo = metododef calcular_metricas_portfolio(pesos, retornos_esperados, cov_matrix, rf_anual=0.0):    pesos = np.array(pesos, dtype=float)    retorno = float(np.dot(pesos, retornos_esperados))    volatilidade = float(np.sqrt(np.dot(pesos.T, np.dot(cov_matrix, pesos))))    sharpe = (retorno - rf_anual) / volatilidade if volatilidade > 1e-8 else np.nan    return retorno, volatilidade, sharpedef pesos_equal_weight(n: int) -> np.ndarray:    return np.ones(n) / ndef otimizar_min_volatilidade(retornos_esperados, cov_matrix, peso_min=0.0, peso_max=1.0):    n = len(retornos_esperados)    pesos_iniciais = np.ones(n) / n    limites = tuple((peso_min, peso_max) for _ in range(n))    restricoes = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}        def objetivo(w):        return np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))        result = minimize(objetivo, pesos_iniciais, method='SLSQP', bounds=limites,                      constraints=restricoes, options={'maxiter': 1000})        if result.success:        return ResultadoOtimizacao(result.x, True, "Convergiu", "GMV_SLSQP")    else:        print(f"   ⚠️ GMV não convergiu: {result.message}")        return ResultadoOtimizacao(pesos_equal_weight(n), False, result.message, "GMV_FALLBACK")def otimizar_max_sharpe(retornos_esperados, cov_matrix, rf_anual=0.0, peso_min=0.0, peso_max=1.0):    n = len(retornos_esperados)    pesos_iniciais = np.ones(n) / n    limites = tuple((peso_min, peso_max) for _ in range(n))    restricoes = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}        def objetivo(w):        ret = np.dot(w, retornos_esperados)        vol = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))        if vol < 1e-8:            return 1e10        return -(ret - rf_anual) / vol        result = minimize(objetivo, pesos_iniciais, method='SLSQP', bounds=limites,                      constraints=restricoes, options={'maxiter': 1000})        if result.success:        return ResultadoOtimizacao(result.x, True, "Convergiu", "MAX_SHARPE_SLSQP")    else:        print(f"   ⚠️ Max Sharpe não convergiu: {result.message}")        return ResultadoOtimizacao(pesos_equal_weight(n), False, result.message, "MAX_SHARPE_FALLBACK")def otimizar_para_retorno_alvo(retornos_esperados, cov_matrix, retorno_alvo, peso_min=0.0, peso_max=1.0):    n = len(retornos_esperados)    pesos_iniciais = np.ones(n) / n    limites = tuple((peso_min, peso_max) for _ in range(n))    restricoes = [        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},        {'type': 'eq', 'fun': lambda x: np.dot(x, retornos_esperados) - retorno_alvo}    ]        result = minimize(lambda w: np.dot(w.T, np.dot(cov_matrix, w)), pesos_iniciais,                      method='SLSQP', bounds=limites, constraints=restricoes, options={'maxiter': 500})        return result.x if result.success else Nonedef calcular_fronteira_eficiente(retornos_esperados, cov_matrix, rf_anual, n_pontos=50,                                  peso_min=0.0, peso_max=1.0):    result_gmv = otimizar_min_volatilidade(retornos_esperados, cov_matrix, peso_min, peso_max)    ret_gmv, vol_gmv, _ = calcular_metricas_portfolio(result_gmv.pesos, retornos_esperados, cov_matrix, rf_anual)        result_max = otimizar_max_sharpe(retornos_esperados, cov_matrix, rf_anual, peso_min, peso_max)    ret_max, vol_max, _ = calcular_metricas_portfolio(result_max.pesos, retornos_esperados, cov_matrix, rf_anual)        ret_topo = np.max(retornos_esperados)    retornos_alvo = np.linspace(ret_gmv, min(ret_topo, ret_max * 1.5), n_pontos)        fronteira_vols = []    fronteira_rets = []        for ret_alvo in retornos_alvo:        pesos = otimizar_para_retorno_alvo(retornos_esperados, cov_matrix, ret_alvo, peso_min, peso_max)        if pesos is not None:            ret, vol, _ = calcular_metricas_portfolio(pesos, retornos_esperados, cov_matrix, rf_anual)            fronteira_vols.append(vol)            fronteira_rets.append(ret)        pontos = sorted(set(zip(fronteira_vols, fronteira_rets)), key=lambda x: x[0])        if pontos:        vols, rets = zip(*pontos)        return np.array(vols), np.array(rets)    return np.array([vol_gmv]), np.array([ret_gmv])print("✅ Funções de otimização carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - MONTE CARLO# =============================================================================def simular_portfolios_monte_carlo(retornos_esperados, cov_matrix, rf_anual,                                   n_portfolios=50000, seed=42,                                   alpha_conc=0.2, alpha_div=1.0,                                   frac_cantos=0.05, frac_sparse=0.15):    np.random.seed(seed)    n_ativos = len(retornos_esperados)        n_cantos = int(n_portfolios * frac_cantos)    n_sparse = int(n_portfolios * frac_sparse)    n_conc = int((n_portfolios - n_cantos - n_sparse) * 0.6)    n_div = n_portfolios - n_cantos - n_sparse - n_conc        pesos_list = []        # Cantos    for _ in range(n_cantos):        w = np.zeros(n_ativos)        idx = np.random.randint(0, n_ativos)        w[idx] = np.random.uniform(0.85, 0.98)        resto = 1.0 - w[idx]        outros = [i for i in range(n_ativos) if i != idx]        if outros:            w[outros] = np.random.dirichlet([0.5] * len(outros)) * resto        pesos_list.append(w)        # Sparse    for _ in range(n_sparse):        w = np.zeros(n_ativos)        k = np.random.randint(3, min(9, n_ativos + 1))        idx_sel = np.random.choice(n_ativos, size=k, replace=False)        w[idx_sel] = np.random.dirichlet([0.5] * k)        pesos_list.append(w)        # Concentrado    pesos_list.extend(np.random.dirichlet([alpha_conc] * n_ativos, size=n_conc))        # Diversificado    pesos_list.extend(np.random.dirichlet([alpha_div] * n_ativos, size=n_div))        pesos = np.array(pesos_list)    retornos = pesos @ retornos_esperados    volatilidades = np.sqrt(np.einsum('ij,jk,ik->i', pesos, cov_matrix, pesos))    sharpes = np.where(volatilidades > 1e-8, (retornos - rf_anual) / volatilidades, np.nan)        return retornos, volatilidades, sharpes, pesosprint("✅ Funções de Monte Carlo carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - TAXA LIVRE DE RISCO (CDI)# =============================================================================def obter_cdi_atual():    try:        url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados/ultimos/1?formato=json"        df = pd.read_json(url)        if df.empty:            return None        valor_str = str(df.loc[0, "valor"]).replace(",", ".")        taxa_diaria = float(valor_str) / 100.0        return float((1.0 + taxa_diaria) ** 252 - 1.0)    except:        return Nonedef obter_cdi_periodo(data_inicio: str, data_fim: str):    try:        di = pd.to_datetime(data_inicio).strftime("%d/%m/%Y")        df_str = pd.to_datetime(data_fim).strftime("%d/%m/%Y")        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados?formato=json&dataInicial={di}&dataFinal={df_str}"                cdi = pd.read_json(url)        if cdi.empty:            return None                cdi["valor"] = cdi["valor"].astype(str).str.replace(",", ".", regex=False)        cdi["valor"] = pd.to_numeric(cdi["valor"], errors="coerce") / 100.0                return float(np.expm1(np.log1p(cdi["valor"].dropna()).mean() * 252))    except:        return Nonedef obter_taxa_livre_risco(modo, fallback, manual=None, data_inicio=None, data_fim=None):    if modo == "manual":        return manual if manual is not None else fallback        if modo == "atual":        rf = obter_cdi_atual()        return rf if rf is not None else fallback        if modo == "media_periodo" and data_inicio and data_fim:        rf = obter_cdi_periodo(data_inicio, data_fim)        if rf is not None:            return rf        rf = obter_cdi_atual()        return rf if rf is not None else fallback        return fallbackprint("✅ Funções de RF carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - VISUALIZAÇÃO# =============================================================================def criar_grafico_fronteira(ret_sim, vol_sim, sharpe_sim, fronteira_vols, fronteira_rets,                            carteiras_especiais, rf_anual, titulo="Fronteira Eficiente"):    fig = go.Figure()        fig.add_trace(go.Scatter(        x=vol_sim * 100, y=ret_sim * 100, mode='markers',        marker=dict(size=2, color=sharpe_sim, colorscale='Viridis',                    colorbar=dict(title='Sharpe', x=1.02), opacity=0.4),        name='Simulados', hovertemplate='Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<extra></extra>'    ))        fig.add_trace(go.Scatter(        x=fronteira_vols * 100, y=fronteira_rets * 100,        mode='lines', line=dict(color='red', width=3), name='Fronteira Eficiente'    ))        estilos = {        'GMV': {'color': 'blue', 'symbol': 'diamond', 'size': 18},        'Max Sharpe': {'color': 'green', 'symbol': 'triangle-up', 'size': 18},        'Carteira Atual': {'color': 'gold', 'symbol': 'star', 'size': 24}    }        for nome, dados in carteiras_especiais.items():        estilo = estilos.get(nome, {'color': 'purple', 'symbol': 'circle', 'size': 15})        pesos = dados.get('pesos', [])        nomes_ativos = dados.get('nomes', [])                if len(pesos) > 0 and len(nomes_ativos) > 0:            idx_sorted = np.argsort(pesos)[::-1]            top_pesos = '<br>'.join([f"{nomes_ativos[i]}: {pesos[i]*100:.1f}%"                                      for i in idx_sorted[:10] if pesos[i] > 0.01])        else:            top_pesos = ''                hover = f"<b>{nome}</b><br>Ret: {dados['retorno']*100:.2f}%<br>Vol: {dados['volatilidade']*100:.2f}%<br>Sharpe: {dados['sharpe']:.3f}<br><br>{top_pesos}"                fig.add_trace(go.Scatter(            x=[dados['volatilidade'] * 100], y=[dados['retorno'] * 100],            mode='markers+text',            marker=dict(size=estilo['size'], color=estilo['color'], symbol=estilo['symbol'],                        line=dict(width=2, color='black')),            text=[nome], textposition='top center', name=nome,            hovertemplate=hover + '<extra></extra>'        ))        fig.add_hline(y=rf_anual * 100, line_dash="dash", line_color="gray",                  annotation_text=f"RF: {rf_anual*100:.2f}%")        fig.update_layout(        title=dict(text=titulo, x=0.5),        xaxis_title="Volatilidade (%)", yaxis_title="Retorno (%)",        template="plotly_white", width=1100, height=700, legend=dict(x=0.02, y=0.98)    )        return figdef criar_heatmap_correlacao(corr_matrix, titulo="Correlação"):    fig = go.Figure(data=go.Heatmap(        z=corr_matrix.values, x=list(corr_matrix.columns), y=list(corr_matrix.index),        colorscale='RdBu', zmid=0, zmin=-1, zmax=1,        hovertemplate='%{x} vs %{y}<br>Corr: %{z:.3f}<extra></extra>'    ))    fig.update_layout(title=dict(text=titulo, x=0.5), width=900, height=800, xaxis=dict(side='top'))    return figdef criar_grafico_barras_pesos(carteiras, nomes_ativos, top_n=15):    fig = go.Figure()    cores = {'GMV': '#636EFA', 'Max Sharpe': '#00CC96', 'Carteira Atual': '#FFA15A'}        for nome_cart, dados in carteiras.items():        pesos = np.array(dados.get('pesos', []))        if len(pesos) == 0:            continue                df_temp = pd.DataFrame({'ativo': nomes_ativos, 'peso': pesos}).nlargest(top_n, 'peso')                fig.add_trace(go.Bar(            name=nome_cart, x=df_temp['ativo'], y=df_temp['peso'] * 100,            marker_color=cores.get(nome_cart, '#AB63FA')        ))        fig.update_layout(        title="Comparativo de Pesos", xaxis_title="Ativo", yaxis_title="Peso (%)",        barmode='group', template="plotly_white", width=1000, height=500    )    return figprint("✅ Funções de visualização carregadas!")

---## 📥 BLOCO 2 - CARREGAMENTO DE DADOS

In [ ]:
# =============================================================================# ETAPA 1: SELECIONAR ARQUIVO EXCEL (CORRIGIDO v2.6)# =============================================================================EXCEL_PATH = selecionar_arquivo_excel_interativo(    diretorio=EXCEL_DIR,    pattern=EXCEL_PATTERN,    arquivo_padrao=ARQUIVO_PADRAO,    recursive=False)

In [ ]:
# =============================================================================# ETAPA 2: CARREGAR METADADOS# =============================================================================print("\n📂 Carregando metadados das abas...")metadados = carregar_metadados_excel(EXCEL_PATH, ativos_excluir=ATIVOS_EXCLUIR)print(f"   Total de abas candidatas: {len(metadados['abas_candidatas'])}")print(f"   Excluídos manualmente: {metadados['ativos_excluidos_manual']}")

In [ ]:
# =============================================================================# ETAPA 3: PERGUNTAR SE DESEJA CARREGAR PESOS (P1)# =============================================================================print("\n" + "=" * 60)print("CARTEIRA ATUAL")print("=" * 60)# Verificar se aba Resumo existetry:    pesos_resumo_temp, status_temp = ler_pesos_resumo_robusto(EXCEL_PATH)    tem_resumo = status_temp == "encontrado"except:    tem_resumo = Falseif tem_resumo:    n_ativos_peso = sum(1 for v in pesos_resumo_temp.values() if v > 0)    soma_pesos = sum(pesos_resumo_temp.values())    print(f"\n📊 Aba 'Resumo' encontrada!")    print(f"   Ativos com peso > 0: {n_ativos_peso}")    print(f"   Soma dos pesos: {soma_pesos:.2%}")        try:        carregar_pesos = input("\n   Deseja carregar os pesos da carteira atual? (s/n) [s]: ").strip().lower()        CARREGAR_CARTEIRA_ATUAL = carregar_pesos == "" or carregar_pesos == "s"    except:        CARREGAR_CARTEIRA_ATUAL = True        print("   (modo não-interativo: carregando pesos)")else:    print("\n⚠️ Aba 'Resumo' não encontrada ou sem coluna de pesos válida.")    CARREGAR_CARTEIRA_ATUAL = Falseprint(f"\n   Carregar carteira atual: {'SIM' if CARREGAR_CARTEIRA_ATUAL else 'NÃO'}")

In [ ]:
# =============================================================================# ETAPA 4: SEPARAR ABAS POR MIN_OBS# =============================================================================print("\n📊 Aplicando filtro MIN_OBS...")abas_ok, abas_excluidas_min_obs = filtrar_por_min_obs(metadados, MIN_OBS)print(f"   Abas que passaram: {len(abas_ok)}")print(f"   Abas excluídas por MIN_OBS: {len(abas_excluidas_min_obs)}")

In [ ]:
# =============================================================================# ETAPA 5: MAPEAMENTO DE PESOS# =============================================================================if CARREGAR_CARTEIRA_ATUAL:    print("\n" + "=" * 60)    print("MAPEAMENTO DE PESOS")    print("=" * 60)        pesos_resumo, _ = ler_pesos_resumo_robusto(EXCEL_PATH)        df_mapa = mapear_pesos_para_abas(        pesos_resumo=pesos_resumo,        abas_candidatas=abas_ok,        abas_excluidas_min_obs=abas_excluidas_min_obs,        threshold_ok=THRESHOLD_OK,        threshold_revisar=THRESHOLD_REVISAR    )        relatorio_mapa = gerar_relatorio_mapeamento(df_mapa, ALERTA_COBERTURA_MINIMA)        # Exibir tabela de mapeamento    print("\n📋 TABELA DE MAPEAMENTO:")    display(df_mapa.sort_values('peso', ascending=False).head(25))        TEM_PESOS = Trueelse:    df_mapa = None    relatorio_mapa = None    TEM_PESOS = False    print("\n⚠️ Prosseguindo sem carteira atual")

In [ ]:
# =============================================================================# ETAPA 6: EXIBIR AUDITORIA (P1)# =============================================================================if TEM_PESOS and relatorio_mapa:    # Criar vetor de pesos    df_no_universo = df_mapa[df_mapa['status'].isin(['OK', 'REVISAR'])].copy()        ATIVOS_OTIMIZACAO_TEMP = abas_ok    pesos_carteira_temp = np.zeros(len(ATIVOS_OTIMIZACAO_TEMP))        for _, row in df_no_universo.iterrows():        aba = row['aba_mapeada']        if aba in ATIVOS_OTIMIZACAO_TEMP:            idx = ATIVOS_OTIMIZACAO_TEMP.index(aba)            pesos_carteira_temp[idx] += row['peso']        soma_pesos = pesos_carteira_temp.sum()    renormalizou = soma_pesos > 0 and abs(soma_pesos - 1.0) > 0.01        # Exibir auditoria    exibir_auditoria_mapeamento(relatorio_mapa, renormalizou, soma_pesos)

In [ ]:
# =============================================================================# ETAPA 7: CARREGAR RETORNOS# =============================================================================print("\n📂 Carregando retornos...")retornos_df = carregar_retornos_das_abas(EXCEL_PATH, abas_ok)ATIVOS_OTIMIZACAO = list(retornos_df.columns)NUM_ATIVOS = len(ATIVOS_OTIMIZACAO)print(f"\n✅ Dados carregados!")print(f"   Ativos no universo: {NUM_ATIVOS}")print(f"   Período: {retornos_df.index.min().strftime('%Y-%m')} a {retornos_df.index.max().strftime('%Y-%m')}")

In [ ]:
# =============================================================================# ETAPA 8: CRIAR VETOR DE PESOS FINAL# =============================================================================if TEM_PESOS and df_mapa is not None:    df_no_universo = df_mapa[df_mapa['status'].isin(['OK', 'REVISAR'])].copy()        pesos_carteira = np.zeros(NUM_ATIVOS)    for _, row in df_no_universo.iterrows():        aba = row['aba_mapeada']        if aba in ATIVOS_OTIMIZACAO:            idx = ATIVOS_OTIMIZACAO.index(aba)            pesos_carteira[idx] += row['peso']        soma_pesos = pesos_carteira.sum()    if soma_pesos > 0 and abs(soma_pesos - 1.0) > 0.01:        print(f"\n   Renormalizando pesos: {soma_pesos:.2%} → 100%")        pesos_carteira = pesos_carteira / soma_pesos        TEM_CARTEIRA_ATUAL = soma_pesos > 0else:    pesos_carteira = None    TEM_CARTEIRA_ATUAL = False

---## 📈 BLOCO 3 - CÁLCULO DE ESTATÍSTICAS

In [ ]:
# =============================================================================# TAXA LIVRE DE RISCO# =============================================================================print("\n" + "=" * 60)print("TAXA LIVRE DE RISCO (CDI)")print("=" * 60)periodo_inicio = retornos_df.index.min().strftime('%Y-%m-%d')periodo_fim = retornos_df.index.max().strftime('%Y-%m-%d')RF_ANUAL = obter_taxa_livre_risco(RF_MODO, RF_FALLBACK, RF_MANUAL, periodo_inicio, periodo_fim)print(f"\n📊 RF anual: {RF_ANUAL*100:.2f}% a.a. (modo: {RF_MODO})")

In [ ]:
# =============================================================================# ESTATÍSTICAS DOS ATIVOS# =============================================================================print("\n" + "=" * 60)print("ESTATÍSTICAS DOS ATIVOS")print("=" * 60)retornos_esperados, cov_matrix, pares_zerados, ativos_floor = calcular_estatisticas_portfolio(    retornos_df, usar_geometrico=USAR_MEDIA_GEOMETRICA,     vol_floor_mensal=VOL_FLOOR_MENSAL, min_overlap=MIN_OVERLAP)if pares_zerados > 0:    print(f"\n⚠️ MIN_OVERLAP: {pares_zerados} pares zerados")if ativos_floor:    print(f"⚠️ VOL_FLOOR aplicado: {ativos_floor[:5]}{'...' if len(ativos_floor) > 5 else ''}")cov_matrix, foi_corrigida = corrigir_matriz_psd(cov_matrix)if foi_corrigida:    print("⚠️ Matriz PSD corrigida")corr_matrix = retornos_df.corr()volatilidades = pd.Series({col: np.sqrt(cov_matrix.loc[col, col]) for col in cov_matrix.columns})print(f"\n{'Ativo':<30} {'Ret.':<10} {'Vol.':<10} {'Sharpe':<8}")print("-" * 60)for ativo in ATIVOS_OTIMIZACAO[:12]:    ret = retornos_esperados[ativo]    vol = volatilidades[ativo]    sharpe = (ret - RF_ANUAL) / vol if vol > 1e-6 else np.nan    print(f"{ativo:<30} {ret*100:>7.2f}% {vol*100:>7.2f}% {sharpe:>7.2f}")if len(ATIVOS_OTIMIZACAO) > 12:    print(f"... e mais {len(ATIVOS_OTIMIZACAO) - 12} ativos")

---## ⚙️ BLOCO 4 - OTIMIZAÇÃO DE PORTFÓLIO

In [ ]:
# =============================================================================# OTIMIZAÇÃO# =============================================================================print("\n" + "=" * 60)print("OTIMIZAÇÃO DE PORTFÓLIO")print("=" * 60)ret_array = retornos_esperados.valuescov_array = cov_matrix.valuesotim_status = {}print("\n🔍 Otimizando GMV...")result_gmv = otimizar_min_volatilidade(ret_array, cov_array, PESO_MIN_ATIVO, PESO_MAX_ATIVO)pesos_gmv = result_gmv.pesosret_gmv, vol_gmv, sharpe_gmv = calcular_metricas_portfolio(pesos_gmv, ret_array, cov_array, RF_ANUAL)otim_status['GMV'] = result_gmvprint("🔍 Otimizando Max Sharpe...")result_max_sharpe = otimizar_max_sharpe(ret_array, cov_array, RF_ANUAL, PESO_MIN_ATIVO, PESO_MAX_ATIVO)pesos_max_sharpe = result_max_sharpe.pesosret_max_sharpe, vol_max_sharpe, sharpe_max_sharpe = calcular_metricas_portfolio(pesos_max_sharpe, ret_array, cov_array, RF_ANUAL)otim_status['Max Sharpe'] = result_max_sharpeprint("🔍 Calculando Fronteira Eficiente...")fronteira_vols, fronteira_rets = calcular_fronteira_eficiente(    ret_array, cov_array, RF_ANUAL, n_pontos=N_PONTOS_FRONTEIRA,    peso_min=PESO_MIN_ATIVO, peso_max=PESO_MAX_ATIVO)print(f"   {len(fronteira_vols)} pontos calculados")if TEM_CARTEIRA_ATUAL:    ret_atual, vol_atual, sharpe_atual = calcular_metricas_portfolio(pesos_carteira, ret_array, cov_array, RF_ANUAL)print("\n📊 STATUS DAS OTIMIZAÇÕES:")for nome, result in otim_status.items():    emoji = "✅" if result.sucesso else "⚠️"    print(f"   {emoji} {nome}: {result.metodo}")

---## 🎲 BLOCO 5 - SIMULAÇÃO MONTE CARLO

In [ ]:
# =============================================================================# SIMULAÇÃO MONTE CARLO# =============================================================================print("\n" + "=" * 60)print(f"SIMULAÇÃO MONTE CARLO ({NUM_PORTFOLIOS:,} portfólios)")print("=" * 60)ret_sim, vol_sim, sharpe_sim, pesos_sim = simular_portfolios_monte_carlo(    ret_array, cov_array, RF_ANUAL,    n_portfolios=NUM_PORTFOLIOS, seed=RANDOM_SEED,    alpha_conc=MC_ALPHA_CONC, alpha_div=MC_ALPHA_DIV,    frac_cantos=MC_FRAC_CANTOS, frac_sparse=MC_FRAC_SPARSE)print(f"\n✅ Simulação concluída!")

---## 📊 BLOCO 6 - VISUALIZAÇÕES

In [ ]:
# =============================================================================# GRÁFICOS# =============================================================================carteiras_especiais = {    'GMV': {'retorno': ret_gmv, 'volatilidade': vol_gmv, 'sharpe': sharpe_gmv,            'pesos': pesos_gmv, 'nomes': ATIVOS_OTIMIZACAO},    'Max Sharpe': {'retorno': ret_max_sharpe, 'volatilidade': vol_max_sharpe,                    'sharpe': sharpe_max_sharpe, 'pesos': pesos_max_sharpe, 'nomes': ATIVOS_OTIMIZACAO}}if TEM_CARTEIRA_ATUAL:    carteiras_especiais['Carteira Atual'] = {        'retorno': ret_atual, 'volatilidade': vol_atual, 'sharpe': sharpe_atual,        'pesos': pesos_carteira, 'nomes': ATIVOS_OTIMIZACAO    }fig_frontier = criar_grafico_fronteira(    ret_sim, vol_sim, sharpe_sim, fronteira_vols, fronteira_rets,    carteiras_especiais, RF_ANUAL, titulo=f"Fronteira Eficiente - PyPortfolio {VERSAO}")fig_frontier.show()

In [ ]:
fig_corr = criar_heatmap_correlacao(corr_matrix, "Matriz de Correlação")fig_corr.show()

In [ ]:
fig_pesos = criar_grafico_barras_pesos(carteiras_especiais, ATIVOS_OTIMIZACAO, top_n=TOP_N_PESOS)fig_pesos.show()

---## 📋 BLOCO 7 - TABELAS ESTRUTURADAS

In [ ]:
# =============================================================================# TABELA DE MÉTRICAS# =============================================================================print("\n" + "=" * 60)print("TABELA DE MÉTRICAS")print("=" * 60)dados_metricas = [    {'Carteira': 'GMV', 'Retorno (%)': ret_gmv * 100, 'Vol (%)': vol_gmv * 100, 'Sharpe': sharpe_gmv},    {'Carteira': 'Max Sharpe', 'Retorno (%)': ret_max_sharpe * 100, 'Vol (%)': vol_max_sharpe * 100, 'Sharpe': sharpe_max_sharpe}]if TEM_CARTEIRA_ATUAL:    dados_metricas.append({'Carteira': 'Carteira Atual', 'Retorno (%)': ret_atual * 100,                            'Vol (%)': vol_atual * 100, 'Sharpe': sharpe_atual})df_metricas = pd.DataFrame(dados_metricas)df_metricas['Retorno (%)'] = df_metricas['Retorno (%)'].round(2)df_metricas['Vol (%)'] = df_metricas['Vol (%)'].round(2)df_metricas['Sharpe'] = df_metricas['Sharpe'].round(3)print("\n")display(df_metricas.set_index('Carteira'))

In [ ]:
# =============================================================================# TABELA DE PESOS# =============================================================================print("\n" + "=" * 60)print("TABELA DE PESOS")print("=" * 60)df_pesos = pd.DataFrame({    'Ativo': ATIVOS_OTIMIZACAO,    'GMV (%)': pesos_gmv * 100,    'Max Sharpe (%)': pesos_max_sharpe * 100})if TEM_CARTEIRA_ATUAL:    df_pesos['Carteira Atual (%)'] = pesos_carteira * 100    df_pesos = df_pesos.sort_values('Carteira Atual (%)', ascending=False)else:    df_pesos = df_pesos.sort_values('Max Sharpe (%)', ascending=False)for col in df_pesos.columns[1:]:    df_pesos[col] = df_pesos[col].round(2)print(f"\n📊 TOP {TOP_N_PESOS} ATIVOS:")display(df_pesos.head(TOP_N_PESOS).reset_index(drop=True))

---## 📋 BLOCO 8 - RESUMO FINAL

In [ ]:
# =============================================================================# RESUMO FINAL# =============================================================================print("\n" + "=" * 70)print(f"RESUMO DA ANÁLISE - PyPortfolio {VERSAO}")print("=" * 70)print(f"\n📊 CONFIGURAÇÃO")print(f"   Arquivo: {Path(EXCEL_PATH).name}")print(f"   Ativos: {NUM_ATIVOS} | MIN_OBS: {MIN_OBS} | MIN_OVERLAP: {MIN_OVERLAP}")print(f"   RF (CDI): {RF_ANUAL*100:.2f}% a.a.")if TEM_PESOS and relatorio_mapa:    print(f"\n📊 MAPEAMENTO")    print(f"   Cobertura: {relatorio_mapa['cobertura']:.1%}")    if relatorio_mapa['alerta_cobertura_baixa']:        print(f"   ⚠️ ALERTA: Cobertura baixa!")print(f"\n📈 CARTEIRAS")print(f"   {'Carteira':<18} {'Retorno':<10} {'Vol':<10} {'Sharpe':<8} {'Status'}")print(f"   {'-'*56}")print(f"   {'GMV':<18} {ret_gmv*100:>7.2f}%  {vol_gmv*100:>7.2f}%  {sharpe_gmv:>6.3f}   {'✅' if otim_status['GMV'].sucesso else '⚠️'}")print(f"   {'Max Sharpe':<18} {ret_max_sharpe*100:>7.2f}%  {vol_max_sharpe*100:>7.2f}%  {sharpe_max_sharpe:>6.3f}   {'✅' if otim_status['Max Sharpe'].sucesso else '⚠️'}")if TEM_CARTEIRA_ATUAL:    print(f"   {'Carteira Atual ⭐':<18} {ret_atual*100:>7.2f}%  {vol_atual*100:>7.2f}%  {sharpe_atual:>6.3f}")print("\n" + "=" * 70)print("✅ Análise concluída!")print("=" * 70)

---## 🗺️ Roadmap de Próximas Versões (Priorizado)### 1. 🔄 Conversão USD→BRL (FX) — Alta Prioridade**Por que importa:** Ativos em USD (ações americanas, cripto) têm retornos em dólares. Sem conversão, a comparação com ativos em BRL fica incorreta.**Complexidade:** Média**Dependências:** Nenhuma**Implementação:**- Baixar série de câmbio USD/BRL (ticker `BRL=X` no Yahoo Finance)- Aplicar fórmula: `r_BRL = (1 + r_USD) * (1 + r_FX) - 1`- Marcar quais ativos foram convertidos---### 2. 📥 Fonte secundária via yfinance + cache — Alta Prioridade**Por que importa:** Quando Excel não disponível, ter alternativa automática.**Complexidade:** Média**Dependências:** Nenhuma**Implementação:**- Hierarquia: Excel → carteira/tickers → DEFAULT_UNIVERSE- Cache local em arquivo pickle/parquet- Fallback transparente com log---### 3. 🎯 Fuzzy matching com rapidfuzz — Média Prioridade**Por que importa:** `SequenceMatcher` é OK, mas `rapidfuzz` (token_set_ratio) é mais robusto para nomes com palavras fora de ordem.**Complexidade:** Baixa**Dependências:** Instalar `rapidfuzz`---### 4. 🏷️ Restrições por classe e por ativo — Média Prioridade**Por que importa:** Carteiras práticas têm limites (ex: máx 10% em cripto, mín 20% em RF).**Complexidade:** Média**Dependências:** Classificação de ativos por tipo---### 5. 📊 Backtesting out-of-sample — Média Prioridade**Por que importa:** Validar se a otimização "funciona" fora da amostra de treino.**Complexidade:** Alta**Dependências:** [1] USD→BRL, [4] Restrições---### 6. 📈 Black-Litterman — Baixa Prioridade**Por que importa:** Incorporar "views" do investidor de forma sistemática.**Complexidade:** Alta**Dependências:** [4] Restrições, [7] Covariância robusta---### 7. 🛡️ Covariância robusta (Ledoit-Wolf) — Média Prioridade**Por que importa:** Com poucos dados, covariância amostral é instável. Shrinkage melhora.**Complexidade:** Média**Dependências:** Nenhuma---### 8. 🖥️ Melhorias de UX/export — Baixa Prioridade**Por que importa:** Facilitar uso e compartilhamento de resultados.**Complexidade:** Baixa**Implementação:** Export CSV/HTML, filtros interativos, themes

---## ✅ Checklist de Validação (v2.6)### P0 - Seletor de Arquivos- [ ] Com 5+ arquivos na pasta, TODOS são listados- [ ] Com arquivo padrão existente, pergunta se quer usar- [ ] Responder "n" permite escolher outro arquivo- [ ] Arquivos temporários (~$) são ignorados- [ ] Mensagem clara se nenhum arquivo encontrado### P0 - Documentação de Parâmetros- [ ] Seção "Parâmetros e Configuração" existe- [ ] Tabela com todos os parâmetros principais- [ ] Cada parâmetro tem: valor padrão, descrição, quando ajustar, trade-off### P1 - Pergunta sobre Pesos- [ ] Pergunta "Deseja carregar pesos?" aparece- [ ] Responder "n" segue sem carteira atual- [ ] Responder "s" carrega e faz mapeamento### P1 - Auditoria de Mapeamento- [ ] Soma de pesos no Resumo exibida- [ ] % mapeado OK, REVISAR, excluído, não mapeado- [ ] ALERTA se cobertura < 60%- [ ] Indica se renormalizou### Geral- [ ] Tabela de métricas exibida após gráficos- [ ] Tabela de pesos exibida e ordenada- [ ] Roadmap no final do notebook